In [6]:
!pip install tensorflow-model-optimization

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot
import numpy as np

# Load the tf_flowers dataset
dataset_name = "tf_flowers"
(ds_train, ds_test), ds_info = tfds.load(dataset_name, split=["train[:80%]", "train[80%:]"], as_supervised=True, with_info=True)

# Preprocessing function
IMG_SIZE = 224

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE)) / 255.0
    return image, label

ds_train = ds_train.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

# Define a simple CNN model
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(ds_info.features["label"].num_classes, activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(ds_train, epochs=5, validation_data=ds_test)

# Apply Magnitude-Based Pruning
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

pruning_params = {
    "pruning_schedule": tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.2, final_sparsity=0.8, begin_step=0, end_step=1000),
}

pruned_model = prune_low_magnitude(model, **pruning_params)

# Compile the pruned model
pruned_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the pruned model for fine-tuning
pruned_model.fit(ds_train, epochs=3, validation_data=ds_test, callbacks=[tfmot.sparsity.keras.UpdatePruningStep()])

# Strip pruning for deployment
pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

# Compile the stripped model again before evaluation
pruned_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]) # Re-compile the stripped model

# Evaluate the pruned model
loss, acc = pruned_model.evaluate(ds_test)
print(f"Pruned model accuracy: {acc:.4f}")


Epoch 1/5
92/92 [==============================] - 11s 86ms/step - loss: 0.5622 - accuracy: 0.7990 - val_loss: 0.3303 - val_accuracy: 0.8856
Epoch 2/5
92/92 [==============================] - 5s 50ms/step - loss: 0.2632 - accuracy: 0.9101 - val_loss: 0.3329 - val_accuracy: 0.8978
Epoch 3/5
92/92 [==============================] - 4s 47ms/step - loss: 0.1649 - accuracy: 0.9489 - val_loss: 0.3777 - val_accuracy: 0.8856
Epoch 4/5
92/92 [==============================] - 4s 45ms/step - loss: 0.1157 - accuracy: 0.9697 - val_loss: 0.4537 - val_accuracy: 0.8638
Epoch 5/5
92/92 [==============================] - 4s 45ms/step - loss: 0.0931 - accuracy: 0.9751 - val_loss: 0.3721 - val_accuracy: 0.8992
Epoch 1/3
92/92 [==============================] - 44s 121ms/step - loss: 0.0761 - accuracy: 0.9782 - val_loss: 0.4312 - val_accuracy: 0.8869
Epoch 2/3
92/92 [==============================] - 9s 101ms/step - loss: 0.5948 - accuracy: 0.7871 - val_loss: 0.4624 - val_accuracy: 0.8311
Epoch 3/3
23/23 